In [1]:
import openpyxl
from openpyxl.drawing.image import Image
from openpyxl.utils import get_column_letter
import requests
import os

In [2]:
def update_and_download_images(file_path, save_directory):
    workbook = openpyxl.load_workbook(filename=file_path)
    sheet = workbook.active

    zinc_ID_ci = 1  # Assuming 'Compound_ID' is in column B (index 1)
    image_url_ci = 7  # Assuming the 'Image_URL' is in column G (index 6)

    for row in sheet.iter_rows(min_row=2, values_only=False):
        zinc_ID = row[zinc_ID_ci].value
        formatted_id = f"ZINC{int(zinc_ID):012d}"
        image_url = f"https://zinc.docking.org/substances/{formatted_id}.png"
        row[image_url_ci].value = image_url

        # Define the save path for each image
        image_file_path = os.path.join(save_directory, f"{zinc_ID}.png")

        # Download the image
        download_image(image_url, image_file_path)

    workbook.save(filename=file_path)

def download_image(image_url, save_path):
    response = requests.get(image_url)
    if response.status_code == 200:
        with open(save_path, 'wb') as f:
            f.write(response.content)
    else:
        print(f"Failed to download image from {image_url}")


In [3]:
def add_images(file_path, save_directory):
    workbook = openpyxl.load_workbook(filename=file_path)
    sheet = workbook.active

    # Remove existing images from the sheet
    if hasattr(sheet, '_images'):
        sheet._images = []

    compound_id_ci = 1
    image_column_letter = 'I'

    for row in sheet.iter_rows(min_row=2, values_only=False):
        compound_id = row[compound_id_ci].value
        if compound_id is not None:
            image_filename = f"{compound_id}.png"
            image_path = os.path.join(save_directory, image_filename)
            # Check if the image file exists for this Compound_ID
            if os.path.exists(image_path):
                # Load and insert the image
                img = Image(image_path)
                
                # Adjust image size if needed
                img.width, img.height = img.width / 2, img.height / 2

                sheet.row_dimensions[row[0].row].height = img.height / 1.3
                column_index = openpyxl.utils.column_index_from_string(image_column_letter)
                sheet.column_dimensions[get_column_letter(column_index)].width = img.width / 8

                # Calculate the cell to anchor the image to
                anchor_cell = f"{image_column_letter}{row[0].row}"
                
                # Add the image to the sheet
                sheet.add_image(img, anchor_cell)

    workbook.save(filename=file_path)

In [6]:
# Define the paths
file_path = '/Users/ethangrimes/Documents/Avner/SIT1_models/na/coi_na08.xlsx'
save_directory = '/Users/ethangrimes/Documents/Avner/SIT1_models/mol_images'

# Ensure the save directory exists
os.makedirs(save_directory, exist_ok=True)

# Call the function to update the sheet and download images
update_and_download_images(file_path, save_directory)

add_images(file_path, save_directory)